In [1]:
import openai
import os
import requests
from dotenv import load_dotenv
from langchain.agents import initialize_agent, Tool
from langchain.llms import OpenAI as LangChainOpenAI
from langchain.agents.agent_types import AgentType
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
import tkinter as tk
from tkinter import scrolledtext
import sounddevice as sd
import numpy as np
import scipy.io.wavfile as wavfile
import tempfile
import pyttsx3  # Text-to-Speech library for speaking out the response

# 1. Load environment variables
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

# 2. Text Preprocessing
def preprocess_text(text):
    text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    text_chunks = text_splitter.split_text(text)
    return text_chunks

# 3. OpenFDA Search
def search_openfda(medicine_name):
    try:
        url = f"https://api.fda.gov/drug/label.json?search=openfda.brand_name:{medicine_name}&limit=1"
        response = requests.get(url)
        data = response.json()
        
        if 'results' in data:
            result = data['results'][0]
            brand_name = result['openfda'].get('brand_name', ['Unknown'])[0]
            manufacturer = result['openfda'].get('manufacturer_name', ['Unknown'])[0]
            usage = result.get('indications_and_usage', ['No usage information'])[0]
            dosage = result.get('dosage_and_administration', ['No dosage information'])[0]
            warnings = result.get('warnings', ['No warnings'])[0]
            
            return {
                "Brand Name": brand_name,
                "Manufacturer": manufacturer,
                "Usage": usage,
                "Dosage": dosage,
                "Warnings": warnings
            }
        else:
            return "No information found in OpenFDA."
    except Exception as e:
        return f"Error fetching from OpenFDA: {e}"

# 4. Chat with GPT
def chat_with_gpt(prompt):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo", 
        messages=[{"role": "system", "content": "You are a medicine assistant."},
                  {"role": "user", "content": prompt}])
    return response['choices'][0]['message']['content']

# 5. LangChain Tool for OpenFDA Search
openfda_tool = Tool(
    name="search_openfda",
    func=search_openfda,
    description="Use this tool to get information about medicines from the OpenFDA database."
)

# 6. Setup Vector Database
def setup_vector_db(texts):
    embeddings = OpenAIEmbeddings()
    vector_store = FAISS.from_texts(texts, embeddings)
    return vector_store

# 7. LangChain Agent
llm = LangChainOpenAI(temperature=0, openai_api_key=openai.api_key)

agent = initialize_agent(
    tools=[openfda_tool],
    llm=llm,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# 8. Speech Recognition using OpenAI Whisper API
def recognize_speech_openai():
    """Record audio from mic and use OpenAI Whisper API."""
    fs = 16000  # Sample rate
    seconds = 5  # Duration of recording
    print("Recording...")
    myrecording = sd.rec(int(seconds * fs), samplerate=fs, channels=1)
    sd.wait()  # Wait until recording is finished
    print("Recording finished. Transcribing...")

    # Save recording temporarily
    with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as f:
        wavfile.write(f.name, fs, (myrecording * 32767).astype(np.int16))
        audio_file = open(f.name, "rb")
        
        # Send to OpenAI Whisper API for transcription
        transcript = openai.Audio.transcribe("whisper-1", audio_file)
        return transcript["text"]

# 9. Text-to-Speech (TTS) using pyttsx3
def speak_text(text):
    engine = pyttsx3.init()
    engine.say(text)
    engine.runAndWait()

# 10. GUI with Tkinter
class MedicineAssistantApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Medicine Assistant")
        self.root.geometry("600x500")
        
        frame = tk.Frame(self.root)
        frame.pack(padx=10, pady=10, expand=True, fill="both")
        
        self.conversation_display = scrolledtext.ScrolledText(frame, wrap=tk.WORD, width=70, height=20, font=("Arial", 12))
        self.conversation_display.pack(pady=5)
        self.conversation_display.config(state=tk.DISABLED)
        
        self.user_input = tk.Entry(frame, width=70, font=("Arial", 12))
        self.user_input.pack(pady=5)
        
        self.send_button = tk.Button(frame, text="Send", font=("Arial", 12), command=self.handle_user_input)
        self.send_button.pack(pady=5)

        self.speech_button = tk.Button(frame, text="Speak", font=("Arial", 12), command=self.handle_speech_input)
        self.speech_button.pack(pady=5)
        
    def display_message(self, message, is_user=False):
        self.conversation_display.config(state=tk.NORMAL)
        if is_user:
            self.conversation_display.insert(tk.END, f"You: {message}\n", "user")
        else:
            self.conversation_display.insert(tk.END, f"Bot: {message}\n", "bot")
        self.conversation_display.config(state=tk.DISABLED)
        self.conversation_display.yview(tk.END)
    
    def handle_user_input(self):
        user_query = self.user_input.get()
        if user_query.lower() in ["exit", "quit"]:
            self.root.quit()
        
        self.display_message(user_query, is_user=True)
        
        if "what is" in user_query.lower() or "tell me about" in user_query.lower():
            medicine_name = user_query.lower().replace("what is", "").replace("tell me about", "").strip()
            response = agent.run(medicine_name)
            self.display_message(response)
        else:
            answer = chat_with_gpt(user_query)
            self.display_message(answer)

        self.user_input.delete(0, tk.END)
        
        # Speak the response
        speak_text(answer)
    
    def handle_speech_input(self):
        try:
            speech_text = recognize_speech_openai()
            self.display_message(speech_text, is_user=True)
            
            # Default answer if something goes wrong
            answer = "I'm sorry, I couldn't process that request."

            if "what is" in speech_text.lower() or "tell me about" in speech_text.lower():
                medicine_name = speech_text.lower().replace("what is", "").replace("tell me about", "").strip()
                response = agent.run(medicine_name)
                self.display_message(response)
                answer = response  # Set the answer variable to the response
            else:
                answer = chat_with_gpt(speech_text)
                self.display_message(answer)
            
            # Speak the response after generating the answer
            speak_text(answer)

        except Exception as e:
            self.display_message(f"Speech recognition failed: {e}")
            answer = "An error occurred while processing the speech input."
            speak_text(answer)  # Speak the error message


# 11. Run the app
if __name__ == "__main__":
    root = tk.Tk()
    app = MedicineAssistantApp(root)
    root.mainloop()


C:\Users\R\AppData\Local\Temp\ipykernel_10672\1419927350.py:78: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = LangChainOpenAI(temperature=0, openai_api_key=openai.api_key)
C:\Users\R\AppData\Local\Temp\ipykernel_10672\1419927350.py:80: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecu

Recording...
Recording finished. Transcribing...
